## Problem 2.

To find the minimum of the function $f(x)$ given by,

\begin{equation}
f(x) = (x - 2)^2 + (y - 2)^2
\end{equation}

Subject to:

\begin{align*}
x + y & \geq 4 \\
2x - y & \geq 0
\end{align*}

Therefore, using the logarithmic penalty method, we obtain the problem:

\begin{equation}
\min_{x,\mu} f(x) + \mu \phi(g(x)) = \min_{x,\mu} \, (x - 2)^2 + (y - 2)^2 - \mu(\log(x + y - 4) + \log(2x - y))
\end{equation}

In this problem, we aim to minimize the function $f(x)$ while satisfying the given constraints by introducing a penalty term controlled by the parameter $f(x)$.

In [2]:
import sympy as sp
import numpy as np
import pandas as pd
from sympy.utilities.lambdify import lambdify

# Define the Newton method for optimization
def Newton(f, x0, ε, J, mu):
    # Define symbolic variables x and y
    x, y = sp.symbols('x y')

    # Calculate the gradient of the objective function
    grad_f = sp.Matrix([sp.diff(f, x), sp.diff(f, y)])

    # Calculate the Hessian matrix (second derivative) of the objective function
    Hess_f = sp.Matrix([[sp.diff(grad_f[i], x), sp.diff(grad_f[i], y)] for i in range(2)])

    # Create lambda functions for numerical evaluation from symbolic expressions
    f_lambda = lambdify((x, y), f, 'numpy')  # Objective function
    grad_lambda = lambdify((x, y), grad_f, 'numpy')  # Gradient
    hessian_lambda = lambdify((x, y), Hess_f, 'numpy')  # Hessian

    # Initialize the parameter vector x with the initial point x0
    x = np.array(x0, dtype=float)
    num_iterations = 0  # Iteration counter

    # List to store optimization data at each iteration
    optimization_data = []

    # Start of the optimization loop
    while num_iterations <= J:
        # Calculate the gradient and Hessian at the current point
        g = np.array(grad_lambda(x[0], x[1]), dtype=float)
        H = np.array(hessian_lambda(x[0], x[1]), dtype=float)

        grad_norm = np.linalg.norm(g)
        fx = f_lambda(x[0], x[1])

        # Append data to the list as a dictionary, separating x into x and y
        optimization_data.append({'Iteration': num_iterations, 'x': x[0], 'y': x[1], 'f(x)': fx, 'Gradient Norm': grad_norm})

        if grad_norm <= ε or num_iterations == J:
            break

        # Update the parameter vector x using the Newton method with the penalized objective function
        x = x - np.reshape(np.dot(np.linalg.inv(H), g), x.shape)
        num_iterations += 1

    # Create a DataFrame to store optimization data
    optimization_data_df = pd.DataFrame(optimization_data)

    return fx, optimization_data_df

# Define symbolic variables x, y, and mu
x, y, μ = sp.symbols('x y mu')

# Define the objective function f(x, y)
f = (x - 2)**2 + (y - 2)**2 - μ*(sp.log(x + y - 4) + sp.log(2*x - y))

# Set tolerance ε, the max number of iterations J, and the initial point x0
ε = 1e-15
J = 100
x0 = [3, 2]

# Set initial large mu value
mu_value = 10**(-10)
results = []
opts = []

# Call the Newton method for optimization
result, optimization_data = Newton(f.subs(μ, mu_value), x0, ε, J, mu_value)

# Print the DataFrame with all the info
print('\n')
print(optimization_data)




    Iteration         x         y          f(x)  Gradient Norm
0           0  3.000000  2.000000  1.000000e+00   2.000000e+00
1           1  2.000000  2.000000  2.154995e-09   6.464992e-01
2           2  2.000000  2.000000  2.085680e-09   3.232496e-01
3           3  2.000000  2.000000  2.016365e-09   1.616248e-01
4           4  2.000000  2.000000  1.947051e-09   8.081240e-02
5           5  2.000000  2.000000  1.877736e-09   4.040620e-02
6           6  2.000000  2.000000  1.808421e-09   2.020309e-02
7           7  2.000000  2.000000  1.739107e-09   1.010154e-02
8           8  2.000000  2.000000  1.669792e-09   5.050748e-03
9           9  2.000000  2.000000  1.600480e-09   2.525334e-03
10         10  2.000000  2.000000  1.531173e-09   1.262588e-03
11         11  2.000000  2.000000  1.461889e-09   6.311357e-04
12         12  2.000000  2.000000  1.392700e-09   3.152514e-04
13         13  2.000000  2.000000  1.323884e-09   1.569951e-04
14         14  2.000001  2.000001  1.256538e-09   7.7

Minimum of $f(x)$ found at $x^* = (2.000005, 2.000005) \rightarrow (2,2)$ 